## ML reproduction

In [1]:
import sys, os, fnmatch, csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from joblib import dump, load

from sklearn.model_selection import train_test_split

#sys.path.insert(0, os.path.dirname(os.getcwd()))

PATH_MODELS = 'C:/Projects/EEG_age_prediction/trained_models'
PATH_DATA_PROCESSED_ML =  'C:/Projects/EEG_explorer/Data/data_processed_ML'

In [2]:

file_names = os.listdir(PATH_DATA_PROCESSED_ML)
file_names

['extracted_features_001_23_jc_mmn36_wk_mmn25_wk.h5',
 'extracted_features_001_29_jc_mmn.h5',
 'extracted_features_001_35_jc_mmn36_wk.h5',
 'extracted_features_002_23_jc_mmn36_wk_mmn25.h5',
 'extracted_features_002_29_jc_mmn25_wk.h5',
 'extracted_features_002_29_jc_mmn36_wk.h5',
 'extracted_features_002_35_jc_mmn36_wk.h5',
 'extracted_features_003_35_jc_mmn36_wk.h5',
 'extracted_features_004_23_mc_mmn36_wk.h5',
 'extracted_features_004_29_mc_mmn25_2_wk.h5',
 'extracted_features_004_29_mc_mmn36_wk.h5',
 'extracted_features_004_29_mc_mmn47_wk.h5',
 'extracted_features_005_11_jc_mmn2.h5',
 'extracted_features_005_17_jc_mmn36_slp_mmn25_slp_mmn47_mixed.h5',
 'extracted_features_005_29_jc_mmn25.h5',
 'extracted_features_005_29_jc_mmn36_2.h5',
 'extracted_features_005_29_jc_mmn36_wk.h5',
 'extracted_features_005_35_jc_mmn36_wk.h5',
 'extracted_features_006_23_mc_mmn25_slp.h5',
 'extracted_features_006_23_mc_mmn36_slp.h5',
 'extracted_features_006_29_mc_mmn36_1_wk.h5',
 'extracted_features_006

['extracted_features_001_23_jc_mmn36_wk_mmn25_wk.h5',
 'extracted_features_001_29_jc_mmn.h5',
 'extracted_features_001_35_jc_mmn36_wk.h5',
 'extracted_features_002_23_jc_mmn36_wk_mmn25.h5',
 'extracted_features_002_29_jc_mmn25_wk.h5',
 'extracted_features_002_29_jc_mmn36_wk.h5',
 'extracted_features_002_35_jc_mmn36_wk.h5',
 'extracted_features_003_35_jc_mmn36_wk.h5',
 'extracted_features_004_23_mc_mmn36_wk.h5',
 'extracted_features_004_29_mc_mmn25_2_wk.h5',
 'extracted_features_004_29_mc_mmn36_wk.h5',
 'extracted_features_004_29_mc_mmn47_wk.h5',
 'extracted_features_005_11_jc_mmn2.h5',
 'extracted_features_005_17_jc_mmn36_slp_mmn25_slp_mmn47_mixed.h5',
 'extracted_features_005_29_jc_mmn25.h5',
 'extracted_features_005_29_jc_mmn36_2.h5',
 'extracted_features_005_29_jc_mmn36_wk.h5',
 'extracted_features_005_35_jc_mmn36_wk.h5',
 'extracted_features_006_23_mc_mmn25_slp.h5',
 'extracted_features_006_23_mc_mmn36_slp.h5',
 'extracted_features_006_29_mc_mmn36_1_wk.h5',
 'extracted_features_006

In [3]:
%%time

from sklearn.model_selection import train_test_split

# Step 1: Get all the files in the output folder
file_names = os.listdir(PATH_DATA_PROCESSED_ML)

# Step 2: Get the full paths of the files (without extensions)
files = [os.path.splitext(os.path.join(PATH_DATA_PROCESSED_ML, file_name))[0] for file_name in fnmatch.filter(file_names, "*.h5")]

# Step 3: Load the features
frames = []

for idx, feature_file in enumerate(files):
    df_features = pd.read_hdf(feature_file + ".h5")
    df_metadata = pd.read_csv(feature_file.replace("extracted_features_", "processed_data_") + ".csv")
    
    # Step 4: Assign labels
    df_features['label'] = df_metadata['age_months'][0]
    
    # Step 5: Assign subject code
    df_features['code'] = df_metadata['code'][0]
    frames.append(df_features)

df = pd.concat(frames) 

# Step 6: List all the unique subject IDs
subject_ids = sorted(list(set(df["code"].tolist())))

IDs_train, IDs_temp = train_test_split(subject_ids, test_size=0.3, random_state=42)
IDs_test, IDs_val = train_test_split(IDs_temp, test_size=0.5, random_state=42)

# Step 7: Split the DataFrames into train, validation and test
df_train = df[df['code'].isin(IDs_train)]
df_val = df[df['code'].isin(IDs_val)]
df_test = df[df['code'].isin(IDs_test)]

feature_names = df.columns.values

MemoryError: Unable to allocate 9.88 GiB for an array with shape (1354, 979751) and data type float64

MemoryError: Unable to allocate 9.88 GiB for an array with shape (1354, 979751) and data type float64

In [4]:
from sklearn.model_selection import train_test_split

IDs_train, IDs_temp = train_test_split(subject_ids, test_size=0.3, random_state=42)
IDs_test, IDs_val = train_test_split(IDs_temp, test_size=0.5, random_state=42)

In [5]:
df_metadata

,code,age_days,age_months,age_years,age_group,cnt_path,cnt_file
0,6,319,10.633333,0.886111,11,NaN,NaN


,code,age_days,age_months,age_years,age_group,cnt_path,cnt_file
0,6,319,10.633333,0.886111,11,NaN,NaN
